In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.parser import parse
import time
import datetime as dt
import re
import pandas_datareader as pdr
import matplotlib.pyplot as plt
import statsmodels.api as sm
# Pretty print all cell's output and not just the last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
#Load the scraped twitter data file after using twitter-scrape.py
header_list = ['author id', 'created_at', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet','ticker_searched']
twitter_data = pd.read_csv('./twitter_data_final.csv',
                   names = header_list)
twitter_data['date'] = pd.to_datetime(twitter_data['created_at'], infer_datetime_format = True)
twitter_data = twitter_data.set_index(['date', 'ticker_searched'])


In [ ]:
#Load S&P500 firms for the time peroid
START_DATE = '2011-03-01'
END_DATE = '2012-03-01'

SP500 = pd.read_csv('./S&P500.txt')
SP500['dtdate'] = pd.to_datetime(SP500['date'], format = "%Y-%m-%d")
SP500 = SP500.loc[(SP500['dtdate'] >= START_DATE) & (SP500['dtdate'] <= END_DATE)].copy()
tickers = []

#since firms have the potential to enter and exit the S&P500 several times a year we want to make sure that they are 
#all incldued
i = SP500.index[SP500['dtdate'] == START_DATE][0]
while i < SP500.index[SP500['dtdate'] == END_DATE][0]:
    current_tickers = (SP500.at[i,'tickers'])
    current_tickers_arr = current_tickers.split(',')
    for tickers_ in current_tickers_arr:
        tickers.append(str.lower('$'+tickers_))
    i = i + 1
    
all_tickers = set(tickers)
tickers = list(all_tickers)
tickers.sort()


In [ ]:
#this function counts the number of times a ticker is present in the body of a tweet
#for example if we have the tweet with the following context:
#"strong day for $AAPL, $AMZN, $MSFT"
#the function would return 3 and save it as 'ticker_count'

def ticker_count(x):
    pattern = re.compile('\W')
    string = re.sub(r'[^A-Za-z0-9 $]+', '', x)
    words = string.split()
    tickers_count = 0
    for words_ in words:
        if words_.lower() in tickers:
            tickers_count += 1
    return tickers_count
twitter_data['ticker_count'] = twitter_data['tweet'].apply(ticker_count)

In [ ]:
#we are now able to remove all data that contains more than 1 ticker
twitter_data = twitter_data.loc[twitter_data['ticker_count'] == 1].copy()

In [ ]:
#this function converts the 'created_at' variable from an object to a date_time
#for whatever reason the twitter API returns the date in two different formats -- this will work for both

def date_fix(x):
    if len(x) == 25:
        x = x[0:19]
        x = pd.to_datetime(x, format = '%Y-%m-%d %H:%M:%S')
    elif len(x) == 19:
        x = x 
        x = pd.to_datetime(x, format = '%Y-%m-%d %H:%M:%S')
    else:
        x = np.nan
    return x
twitter_data['fixed_dt'] = twitter_data['created_at'].apply(date_fix)

In [ ]:
#this function assigns the 'trading_day' for the tweet
#1. comments made after hours are considered for the next trading day
#2. comments made on Sunday are considered for the next trading day (monday)
#3. comments made on Saturday are considered for the next trading day (monday)

def trading_day(x):
    if x.hour > 16:
        x = x + pd.DateOffset(1)
    if x.weekday() == 6:
        x = x + pd.DateOffset(1)
    if x.weekday() == 5:
        x = x + pd.DateOffset(2)
    return x
twitter_data['trading_day'] = twitter_data['fixed_dt'].apply(trading_day)

In [ ]:
#we load the harvard 4 sentiment dictonary
#we make two dictionarys 
#1. words that are attributed with positive sentiment
#2. words that are attributed with negative sentiment

dictionary = pd.read_csv('./dictionary.csv', low_memory=False)
dictionary['Entry'] = dictionary['Entry'].str.lower()
dictionary['Entry'] = dictionary.Entry.str.replace('[^a-zA-Z]',r'',regex=True)
dictionary = dictionary.loc[dictionary['Source'] == 'H4'].copy()
pos = dictionary.loc[dictionary['Positiv'] == 'Positiv'][['Positiv', 'Entry']]
pos = pos.set_index(pos['Entry'])
pos = pos.drop_duplicates()
pos_words = pos.to_dict('index')
neg = dictionary.loc[dictionary['Negativ'] == 'Negativ'][['Negativ', 'Entry']]
neg = neg.set_index(neg['Entry'])
neg = neg.drop_duplicates()
neg_words = neg.to_dict('index')

In [ ]:
#now we count the number of pos/neg sentiment words that exist in each tweet

def pos_sentiment(x):
    words = x.split()
    pos_count = 0
    for words_ in words:
        if words_.lower() in pos_words:
            pos_count += 1
    return pos_count
def neg_sentiment(x):
    words = x.split()
    neg_count = 0
    for words_ in words:
        if words_.lower() in neg_words:
            neg_count += 1
    return neg_count
twitter_data['pos_sentiment'] = twitter_data['tweet'].apply(pos_sentiment)
twitter_data['neg_sentiment'] = twitter_data['tweet'].apply(neg_sentiment)

#we also need the number of words in each tweet
twitter_data['words_count'] = twitter_data['tweet'].str.split().str.len()

In [ ]:
#load market data and make the dates match the format of twitter_data['trading_day']
market_data = pd.read_csv('./hsvtcapxk3srzvtd.csv', low_memory = False)
market_data['datadate'] = pd.to_datetime(market_data['date'], format = "%Y/%m/%d").copy()


In [ ]:
#merge the twitter_data and market_data by date and firm

market_data['just_date'] = market_data['datadate'].dt.date
twitter_data['trading_ddate'] = twitter_data['trading_day'].dt.date
merge1 = pd.merge(twitter_data, market_data, 
                                how = 'left',
                                left_on = ['trading_ddate', 'ticker_searched'],
                                right_on = ['just_date', 'TICKER']
                                )

In [ ]:
#remove data that we will not be using 
merge1 = merge1[['TICKER', 'pos_sentiment', 'neg_sentiment', 'trading_ddate', 'RET', 'words_count']].copy()
merge1 = merge1.set_index(['trading_ddate', 'TICKER']).copy()
merge1 = merge1.sort_index(axis=0)
merge1 = merge1.dropna().copy()

In [ ]:
#count the number of pos/neg sentiment on a per day/per firm basis
pos_sentiment_daily = merge1.groupby([merge1.index.get_level_values(0), merge1.index.get_level_values(1), merge1['RET']])['pos_sentiment'].sum().to_frame()
neg_sentiment_daily = merge1.groupby([merge1.index.get_level_values(0), merge1.index.get_level_values(1), merge1['RET']])['neg_sentiment'].sum().to_frame()

#determine the number of total words and tweets per day/per firm
daily_word_count = merge1.groupby([merge1.index.get_level_values(0), merge1.index.get_level_values(1), merge1['RET']])['words_count'].sum().to_frame()
tweet_count = merge1.groupby([merge1.index.get_level_values(0), merge1.index.get_level_values(1), merge1['RET']])['neg_sentiment'].count().to_frame()
total_tweets = merge1.groupby([merge1.index])

#merge dataframes
sentiment2 = pos_sentiment_daily.merge(neg_sentiment_daily, left_index = True, right_index = True)
sentiment1 = sentiment2.merge(daily_word_count, left_index = True, right_index = True)

tweet_count['count'] = tweet_count['neg_sentiment']
tweet_count = tweet_count.drop(columns = 'neg_sentiment')
sentiment = sentiment1.merge(tweet_count, left_index = True, right_index = True) 
sentiment['total_sentiment'] = sentiment['pos_sentiment'] + sentiment['neg_sentiment']

#calcualte the emotional valence variables
sentiment['neg1'] = (sentiment['neg_sentiment']/sentiment['words_count'])
sentiment['pos1'] = ((sentiment['pos_sentiment'] - sentiment['neg_sentiment']) / (sentiment['total_sentiment'])).fillna(0)
sentiment['pos2'] = np.log10( (1 + sentiment['pos_sentiment']) / (1 + sentiment['neg_sentiment']))


In [ ]:
#load the CRSP/Compustat data
comp_data = pd.read_csv('./comp_data.csv',
                           low_memory = False)
comp_data2 = comp_data.loc[comp_data['exchg'] == 11]



In [ ]:
#reset sentiment index for merge
sentiment = sentiment.reset_index()

#merge sentiment and comp_data
merge2 = pd.merge(sentiment, comp_data, 
                                how = 'left',
                                left_on = ['TICKER'],
                                right_on = ['tic']
                 )

#the trading_ddate variable will occasionally revert to an object data-type, we need it to be datetime data-type
merge2['trading_ddate'] = pd.to_datetime(merge2['trading_ddate'])

In [ ]:
#calculate the book-to-market for each firm
merge2['book_to_market'] = (merge2['at'] - merge2['lt'])/(merge2['mkvalt'])

#calculate the book-to-market quartiles for constructing the Fama French portfolio
mkt_value_median = comp_data2['mkvalt'].median()
comp_data2 = comp_data2.dropna().copy()
comp_data2['book_to_market'] = (comp_data2['at'] - comp_data2['lt'])/(comp_data2['mkvalt'])
book_to_market_quartiles = comp_data2['book_to_market'].quantile(q=[.3,.7])

In [ ]:
#load Fama French data
kf_data = pd.read_csv('./6_Portfolios_2x3_daily.csv',
                      low_memory = False,
                     nrows=25010)
#fix column name
kf_data.rename(columns = {'Unnamed: 0':'date'}, inplace = True)
kf_data = kf_data.dropna().copy()

#make sure date matches format from merge2
kf_data['dtdate'] = pd.to_datetime(kf_data['date'], format = '%Y%m%d')

#convert all percentages to fractions
kf_data['SMALL LoBM'] = kf_data['SMALL LoBM']/100
kf_data['ME1 BM2'] = kf_data['ME1 BM2']/100
kf_data['SMALL HiBM'] = kf_data['SMALL HiBM']/100
kf_data['BIG LoBM'] = kf_data['BIG LoBM']/100
kf_data['ME2 BM2'] = kf_data['ME2 BM2']/100
kf_data['BIG HiBM'] = kf_data['BIG HiBM']/100


In [ ]:
#merge with Fama French data
merge3 = pd.merge(merge2, kf_data, 
                                how = 'left',
                                left_on = ['trading_ddate'],
                                right_on = ['dtdate']
                                )

In [ ]:
#remove all data that we won't use
final_data = merge3[['trading_ddate', 'TICKER', 'RET','total_sentiment', 'neg1', 'pos1', 'pos2', 'book_to_market', 'mkvalt', 'SMALL LoBM', 'ME1 BM2', 'SMALL HiBM', 'BIG LoBM', 'ME2 BM2', 'BIG HiBM', 'count']]

In [ ]:
#CRSP has different error codes for dates without returns we need to remove this data and convert to float.
final_data.loc[final_data['RET'] == 'C', 'RET'] = None
final_data.loc[final_data['RET'] == 'A', 'RET'] = None
final_data.loc[final_data['RET'] == 'B', 'RET'] = None
final_data.loc[final_data['RET'] == 'D', 'RET'] = None
final_data.loc[final_data['RET'] == 'E', 'RET'] = None
final_data = final_data.dropna().copy()
final_data['RET'] = final_data['RET'].astype('float64')


In [ ]:
#organize the firms based on their book-to-market and size (6 FF portfolios)
#calculate the abnormal return of each firm
#this is done for all 6 FF portfolios
df_SMALL_LOBM = final_data.loc[(final_data['book_to_market'] < book_to_market_quartiles.iloc[0]) & (final_data['mkvalt'] < mkt_value_median)]
df_SMALL_LOBM['abnormal_ret'] = df_SMALL_LOBM['RET'] - df_SMALL_LOBM['SMALL LoBM']

df_SMALL_BM2 = final_data.loc[(final_data['book_to_market'] < book_to_market_quartiles.iloc[1]) & (final_data['book_to_market'] > book_to_market_quartiles.iloc[0]) & (final_data['mkvalt'] < mkt_value_median)]
df_SMALL_BM2['abnormal_ret'] = df_SMALL_BM2['RET'] - df_SMALL_BM2['ME1 BM2']

df_SMALL_HiBM = final_data.loc[(final_data['book_to_market'] > book_to_market_quartiles.iloc[1]) & (final_data['mkvalt'] < mkt_value_median)]
df_SMALL_HiBM['abnormal_ret'] = df_SMALL_HiBM['RET'] - df_SMALL_HiBM['SMALL HiBM']

df_BIG_LOBM = final_data.loc[(final_data['book_to_market'] < book_to_market_quartiles.iloc[0]) & (final_data['mkvalt'] > mkt_value_median)]
df_BIG_LOBM['abnormal_ret'] = df_BIG_LOBM['RET'] - df_BIG_LOBM['BIG LoBM']

df_BIG_BM2 = final_data.loc[(final_data['book_to_market'] < book_to_market_quartiles.iloc[1]) & (final_data['book_to_market'] > book_to_market_quartiles.iloc[0]) & (final_data['mkvalt'] > mkt_value_median)]
df_BIG_BM2['abnormal_ret'] = df_BIG_BM2['RET'] - df_BIG_BM2['ME2 BM2']

df_BIG_HiBM = final_data.loc[(final_data['book_to_market'] > book_to_market_quartiles.iloc[1]) & (final_data['mkvalt'] > mkt_value_median)]
df_BIG_HiBM['abnormal_ret'] = df_BIG_HiBM['RET'] - df_BIG_HiBM['BIG HiBM']

#combine all the portfolios
final_df = pd.concat([df_SMALL_LOBM, df_SMALL_BM2, df_SMALL_HiBM, df_BIG_LOBM, df_BIG_BM2, df_BIG_HiBM])

#index by date
final_df.set_index(['trading_ddate'])
final_df.sort_index(inplace = True)

In [ ]:
#having percentage returns makes calculating cum returns easier
final_df['abnormal_ret_plus_1'] = final_df['abnormal_ret'] + 1

#calculate the previous dat reuturn for Control 1 var
final_df['abnormal_ret_1d_prior'] = final_df.groupby('TICKER')['abnormal_ret'].shift(1)

#calculate the cumulative returns for each ticker
final_df['cum_prod'] = final_df.groupby('TICKER')['abnormal_ret_plus_1'].cumprod()

#calculate the next 10 day returns
final_df['abnormal_ret_next_10days'] = final_df.groupby('TICKER')['cum_prod'].shift(-9) / final_df['cum_prod']
final_df['abnormal_ret_next_10days'] = np.power(final_df['abnormal_ret_next_10days'], 1/10) - 1

#calculate the returns for the previous month without the previous dat for Control 2 var
final_df['abnormal_ret_30d_to_2d_prior'] = final_df.groupby('TICKER')['cum_prod'].shift(1) / final_df.groupby('TICKER')['cum_prod'].shift(29)
final_df['abnormal_ret_30d_to_2d_prior'] = final_df['abnormal_ret_30d_to_2d_prior'] - 1

#calculate previous dat returns
final_df['previous_day_ret'] = (1/final_df.groupby('TICKER')['cum_prod'].shift(1)) * final_df['cum_prod']
final_df['previous_day_ret'] = final_df.groupby('TICKER')['previous_day_ret'].shift(1)
final_df['previous_day_ret'] = final_df['previous_day_ret'] -1

#calculate next dat returns
final_df['abnormal_ret_next_day'] = final_df.groupby('TICKER')['abnormal_ret'].shift(-1)

#since we have rolling returns we will not have data for all time peroids, so we need to drop rows with empty values
final_df = final_df.dropna()

#drop any duplicates with same ticker and trading date
final_df = final_df.drop_duplicates(subset = ['TICKER', 'trading_ddate'], keep = 'first').copy()

In [ ]:
#we can now run our OLS regression

x1 = ['pos1', 'abnormal_ret_30d_to_2d_prior', 'previous_day_ret']
x2 = ['pos2', 'abnormal_ret_30d_to_2d_prior', 'previous_day_ret']
x3 = ['neg1', 'abnormal_ret_30d_to_2d_prior', 'previous_day_ret']
y1 = 'abnormal_ret'
y2 = 'abnormal_ret_next_day'
y3 = 'abnormal_ret_next_10days'

y = final_df[y1]
x = final_df[x1]

x = sm.add_constant(x)

res = sm.OLS(y,x).fit()
res

#print(res.summary())
#print(res.rsquared_adj)
#print(res.params)
#print(res.pvalues)
#print(res.bse)

In [ ]:
print(res.summary())